# Ler arquivo txt

In [1]:
with open("input.txt", "r") as tf:
    lines = tf.read().split(',')
    
list_words = lines

In [2]:
list_words = list_words[0].split('\n')

# Configura Spark

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,length,lower

In [4]:
%%time

spark = SparkSession.builder.appName("Word Count").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

CPU times: user 130 ms, sys: 298 ms, total: 428 ms
Wall time: 37.8 s


# 1) Conta Palavras

In [5]:
# Cria lista de dicionarios para gerar o dataframe
list_dict = []
for name in list_words:
    list_dict.append(dict(words = name))

In [ ]:
%%time
# Cria o Dataframe

df = spark.createDataFrame(list_dict)

CPU times: user 8.6 s, sys: 42.8 ms, total: 8.64 s
Wall time: 10.9 s


In [ ]:
%%time
# Conta quantidade de linhas do dataframe
df.count()

CPU times: user 3.46 ms, sys: 95 µs, total: 3.55 ms
Wall time: 2.13 s


1048577

# 2) Quantas vezes cada palavra se repete

In [ ]:
%%time
df = df.groupBy("words").count().orderBy("count",ascending=False)

CPU times: user 2.77 ms, sys: 0 ns, total: 2.77 ms
Wall time: 72.5 ms


In [ ]:
%%time
df = df.withColumn("words",lower(col("words")))

CPU times: user 2.07 ms, sys: 25 µs, total: 2.09 ms
Wall time: 9.38 ms


In [ ]:
%%time
tamanho_df = df.count()

CPU times: user 3.35 ms, sys: 0 ns, total: 3.35 ms
Wall time: 681 ms


In [ ]:
%%time
df.show(tamanho_df)

+------------+-----+
|       words|count|
+------------+-----+
|   Faculdade|44130|
|          de|43978|
|      Victor|43965|
|       Pedro|43926|
|        Gama|43923|
|       Lucas|43899|
|     Sistema|43858|
|    Carvalho|43812|
|        João|43786|
|    Software|43731|
|Distribuidos|43721|
| Programação|43718|
|       Silva|43699|
| Laboratorio|43688|
|      Samuel|43678|
|         UnB|43632|
|   Paralelos|43574|
|         FGA|43569|
|          do|43528|
|      Santos|43484|
|     Materia|43468|
|         dos|43334|
|  Engenharia|43326|
|        PSPD|43149|
|            |    1|
+------------+-----+

CPU times: user 2.43 ms, sys: 68 µs, total: 2.49 ms
Wall time: 787 ms


# 3) Contar quantas palavras começam com as letras S,P,R

In [ ]:
%%time
print(f"Comecam com P = {df.filter(df.words.startswith('p')).agg({'count':'sum'}).collect()[0][0]}")
print(f"Comecam com S = {df.filter(df.words.startswith('s')).agg({'count':'sum'}).collect()[0][0]}")
print(f"Comecam com R = {df.filter(df.words.startswith('r')).agg({'count':'sum'}).collect()[0][0]}")

Comecam com P = 174367
Comecam com S = 218450
Comecam com R = None
CPU times: user 11.9 ms, sys: 4.1 ms, total: 16 ms
Wall time: 1.33 s


# 4) Contar quantas palavras tem tamanho 6,8,11

In [ ]:
%%time
print(f"Tamanho 6 = {df.filter(length(col('words')) == 6).agg({'count':'sum'}).collect()[0][0]}")
print(f"Tamanho 8 = {df.filter(length(col('words')) == 8).agg({'count':'sum'}).collect()[0][0]}")
print(f"Tamanho 11 = {df.filter(length(col('words')) == 11).agg({'count':'sum'}).collect()[0][0]}")

Tamanho 6 = 131127
Tamanho 8 = 87543
Tamanho 11 = 87406
CPU times: user 15.1 ms, sys: 31 µs, total: 15.1 ms
Wall time: 1.33 s
